In [1]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.docstore.document import Document
from nltk.corpus import wordnet
import torch

# Ensure NLTK resources are downloaded
import nltk
nltk.download('wordnet')

# Authenticate with the Kaggle API
api = KaggleApi()
api.authenticate()


/home/nealsharma/.local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nealsharma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import os
import shutil

# Path to Hugging Face cache
cache_dir = os.path.expanduser("~/.cache/huggingface")

# Check if the cache directory exists and delete it
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"Hugging Face cache at {cache_dir} has been cleared.")
else:
    print("Hugging Face cache is already cleared.")


Hugging Face cache at /home/nealsharma/.cache/huggingface has been cleared.


In [3]:
# Prune Git LFS objects to free up space
!git -C /home/nealsharma/llm/Mistral-7B-v0.1 lfs prune


prune: 5 local object(s), 5 retained, done.                                     


In [4]:
# Download the "All The News" dataset from Kaggle
api.dataset_download_files('asad1m9a9h6mood/news-articles', path='data/', unzip=True)

# Load and process the dataset
file_path = 'data/Articles.csv'  # Adjust the path based on dataset name
df = pd.read_csv(file_path, encoding='ISO-8859-1')


Dataset URL: https://www.kaggle.com/datasets/asad1m9a9h6mood/news-articles


In [5]:
# Convert documents into the required format for Chroma 
document_list = [
    Document(
        page_content=row['Article'],
        metadata={
            'date': row['Date'],
            'heading': row['Heading'],
            'news_type': row['NewsType']
        }
    )
    for _, row in df.iterrows()
]

# NOTE Truncate document list because it takes too long to process
max_length = 3
if len(document_list) > max_length:
    document_list = document_list[:max_length]


In [6]:
# Initialize the SentenceTransformer and Chroma vectorstore
# Embed documents using SentenceTransformer

embedding_function = SentenceTransformerEmbeddings(model_name = 'all-MiniLM-L6-v2')

# Create a vector store using Chroma
vector_store = Chroma.from_documents(document_list, embedding_function)


/tmp/ipykernel_2578/1247450238.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name = 'all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/nealsharma/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Function to expand the query using WordNet
def expand_query_with_synonyms(query):
    synonyms = set()
    for word in query.split():
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
    expanded_query = query + " " + " ".join(synonyms)
    return expanded_query

In [8]:
# Start of 2.3

# Helper function to dynamically retrieve more information
def dynamic_retrieval(query, docs, vector_store):
    """
    Function to dynamically retrieve more documents based on the query and the current documents.
    This allows the model to ask for more information if necessary during generation.
    
    Args:
    - query: The current query or prompt being used.
    - docs: List of documents already retrieved, to avoid redundancy.
    - vector_store: The vector store used to perform the retrieval.
    
    Returns:
    - List of newly retrieved documents.
    """
    # Perform dynamic retrieval using vector_store and filtering out already retrieved documents
    new_docs = vector_store.similarity_search(query, top_k=10)  # Retrieve top 10 similar documents
    
    # Filter out already retrieved documents by comparing metadata (e.g., 'heading')
    retrieved_docs = {doc.metadata['heading'] for doc in docs}  # Adjust to your unique document key
    unique_new_docs = [doc for doc in new_docs if doc.metadata['heading'] not in retrieved_docs]
    
    return unique_new_docs



In [9]:
# Function to iteratively generate text and dynamically retrieve more info if needed
def generate_with_dynamic_retrieval(prompt, vector_store, model, tokenizer, max_iterations=3):
    """
    Generate a response while dynamically retrieving more information as needed.
    The model will iteratively request more documents if gaps or ambiguities are detected.
    """
    expanded_query = expand_query_with_synonyms(prompt)
    current_docs = vector_store.similarity_search(expanded_query, k=5)  # Initial retrieval
    generation = ""
    context = " ".join([doc.page_content for doc in current_docs[:2]])  # Limiting to top 2 documents
    
    for i in range(max_iterations):
        # Tokenize the current context
        inputs = tokenizer(context, return_tensors="pt")
        
        # Generate text using the loaded model
        with torch.no_grad():  # Disables gradient computation for faster inference
            outputs = model.generate(
                inputs['input_ids'],
                #attention_mask=inputs['attention_mask'],
                max_new_tokens=50,  # Generate up to 100 new tokens
                do_sample=True,      # Enable sampling for more varied results
                top_k=50,            # Limit the sampling to the top 50 tokens
                temperature=0.7,      # Control the randomness (higher = more random)
                length_penalty=1.5  # Discourages very long outputs
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append generated text to the ongoing generation
        generation += generated_text
        
        # Check if there are ambiguities or gaps in the generated text (trigger more retrieval)
        if "need more information" in generated_text or "clarify" in generated_text:
            # Dynamically retrieve more documents based on the generated content
            new_docs = dynamic_retrieval(generated_text, current_docs, vector_store)
            current_docs += new_docs  # Add newly retrieved documents to the context
            context += " " + " ".join([doc.page_content for doc in new_docs[:2]])  # Update the context
        else:
            break  # Exit if no more information is needed

    return generation


In [10]:
# Expand the initial query
query = "Tell me about what the news in Karachi is  mostly about"

In [11]:
# Set the correct model path
model_path = "/home/nealsharma/llm/Mistral-7B-v0.1"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [ ]:
# Generate text with dynamic retrieval
response = generate_with_dynamic_retrieval(query, vector_store, model, tokenizer)
print(response)


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
/home/nealsharma/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.5` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
